# Week 4 Project: Investor Risk Preference

## Goal
Not all investors are the same. Some strictly avoid risk, while others chase returns.
In this project, we explicitly model **Risk Aversion ($\lambda$)**.

We will:
1.  Generate a universe of feasible portfolios (Brute Force).
2.  Define a **Utility Function**: $U = E[R] - \frac{1}{2} \lambda \sigma^2$
3.  Observe how the "Optimal Portfolio" shifts as $\lambda$ changes.
4.  Visualize the transition from Risk-Seeking to Risk-Averse allocations.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Import Data (Reuse from Week 2)
TICKERS = ["GAIL.NS", "SILVERBEES.NS", "TATAPOWER.NS"]
print(f"Downloading data for: {TICKERS}")
df = yf.download(TICKERS, period="3y")['Close']
df.dropna(inplace=True)
df.head()

In [ ]:
# 2. Compute Statistics
# Log Returns
log_returns = np.log(df / df.shift(1)).dropna()

# Annualize Stats (Assuming 252 trading days)
TRADING_DAYS = 252
mu = log_returns.mean() * TRADING_DAYS
cov_matrix = log_returns.cov() * TRADING_DAYS

print("Expected Annual Returns:\n", mu)
print("\nAnnualized Covariance Matrix:\n", cov_matrix)

In [ ]:
# 3. Generate Feasible Portfolios (Brute Force)
# We generate thousands of random weight combinations to represent the "Feasible Region"

num_portfolios = 5000
results = []
np.random.seed(42)

for i in range(num_portfolios):
    # Random weights that sum to 1
    weights = np.random.random(len(TICKERS))
    weights /= np.sum(weights)
    
    # Portfolio stats
    ret = np.dot(weights, mu)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    # Store details
    # Columns: [Return, Volatility, W1, W2, W3...]
    portfolio_data = [ret, vol] + list(weights)
    results.append(portfolio_data)

columns = ['Return', 'Volatility'] + TICKERS
portfolio_df = pd.DataFrame(results, columns=columns)
print("Generated", num_portfolios, "portfolios.")
portfolio_df.head()

## 4. Modeling Risk Preference ($\lambda$)
We use the Mean-Variance Utility Function:
$$ U = E[R] - \lambda \sigma^2 $$

*   $\lambda$ (Lambda) represents **Risk Aversion**.
*   **Low $\lambda$** (e.g., 0): Investor only cares about Returns (Aggressive).
*   **High $\lambda$** (e.g., 10): Investor hates variance (Conservative).

We will iterate through different $\lambda$ values and pick the portfolio with the **highest Utility** for each.

In [ ]:
lambdas = np.linspace(0, 10, 50)  # Range of risk aversion from 0 to 10
optimal_allocations = []

for lam in lambdas:
    # Compute Utility for ALL 5000 portfolios for this specific lambda
    # Utility = Return - lambda * Variance
    # Note: Variance = Volatility^2
    utilities = portfolio_df['Return'] - lam * (portfolio_df['Volatility'] ** 2)
    
    # Find the single portfolio with the Maximum Utility
    best_idx = utilities.argmax()
    best_portfolio = portfolio_df.iloc[best_idx]
    
    # Record the Lambda and the Weights of this best portfolio
    record = {
        'Lambda': lam,
        'Best_Return': best_portfolio['Return'],
        'Best_Volatility': best_portfolio['Volatility']
    }
    for ticker in TICKERS:
        record[ticker] = best_portfolio[ticker]
    
    optimal_allocations.append(record)

optimal_df = pd.DataFrame(optimal_allocations)
optimal_df.head()

## 5. Visualizations

In [ ]:
# Plot 1: How Allocation Changes as Risk Aversion Increases
plt.figure(figsize=(10, 6))
plt.stackplot(optimal_df['Lambda'], 
              [optimal_df[t] for t in TICKERS], 
              labels=TICKERS, alpha=0.8)
plt.xlabel("Risk Aversion ($\lambda$)")
plt.ylabel("Portfolio Weight")
plt.title("Asset Allocation vs. Risk Preference")
plt.legend(loc='upper right')
plt.margins(0, 0)
plt.show()

In [ ]:
print("Observation:")
print("- At Lambda = 0 (Risk Neutral), we are 100% in the asset with the Highest Return.")
print("- As Lambda increases (Risk Averse), we shift towards safer assets (lower volatility) or diversified mixes.")

In [ ]:
# Plot 2: Mapping Optimal Portfolios on the Efficient Frontier
plt.figure(figsize=(10, 6))

# 1. The Cloud (All Feasible Portfolios)
plt.scatter(portfolio_df['Volatility'], portfolio_df['Return'], 
            c='lightgray', s=5, label='Feasible Portfolios')

# 2. The Path of Optimal Portfolios
plt.plot(optimal_df['Best_Volatility'], optimal_df['Best_Return'], 
         'r-o', linewidth=2, label='Optimal Path (Increasing $\lambda$)')

# Annotate start and end
plt.text(optimal_df.iloc[0]['Best_Volatility'], optimal_df.iloc[0]['Best_Return'], 
         ' $\lambda=0$ (Max Ret)', fontsize=9, verticalalignment='bottom')
plt.text(optimal_df.iloc[-1]['Best_Volatility'], optimal_df.iloc[-1]['Best_Return'], 
         ' $\lambda=10$ (Min Risk)', fontsize=9, verticalalignment='top')

plt.xlabel("Risk (Volatility)")
plt.ylabel("Expected Return")
plt.title("Optimal Portfolios on the Risk-Return Plane")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()